In [ ]:
!pip install pandas numpy sentence-transformers openai tiktoken langchain chromadb langchain-community faiss-cpu gradio langchain-openai

In [ ]:
# kết nối với gg drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Phần tiền xử lý dữ liệu tối ưu
import json
import pandas as pd
import numpy as np
import re
import pickle
import os
from sentence_transformers import SentenceTransformer
from google.colab import files  # Giữ lại cho môi trường Colab

# Hàm đọc dữ liệu từ file
def load_data(file_path):
    """Đọc dữ liệu từ file JSON"""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        print(f"Đã đọc {len(data)} mục từ file {file_path}")
        return data
    except Exception as e:
        print(f"Lỗi khi đọc file: {str(e)}")
        return []

# Hàm làm sạch văn bản
def clean_text(text):
    """Làm sạch văn bản"""
    if not isinstance(text, str) or pd.isna(text):
        return ""
    # Loại bỏ ký tự đặc biệt, giữ lại dấu câu cơ bản
    text = re.sub(r'[^\w\s.,?!;:()[\]{}"\'-]', ' ', text)
    # Chuẩn hóa khoảng trắng
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

# Hàm xử lý và làm sạch dữ liệu
def process_data(data):
    """Xử lý và làm sạch dữ liệu"""
    clean_data = []

    for item in data:
        new_item = {}
        if 'cau_hoi' in item and 'cau_tra_loi' in item:
            new_item['question'] = clean_text(item.get('cau_hoi', ''))
            new_item['answer'] = clean_text(item.get('cau_tra_loi', ''))
        elif 'cau_tra_loi' in item:
            # Xử lý các mục chỉ có câu trả lời
            new_item['question'] = ""
            new_item['answer'] = clean_text(item.get('cau_tra_loi', ''))

        if new_item.get('answer'):  # Thêm vào nếu có câu trả lời
            clean_data.append(new_item)

    print(f"Số dòng dữ liệu sau khi làm sạch: {len(clean_data)}/{len(data)}")
    return clean_data

# Hàm hiển thị thống kê dữ liệu
def show_data_stats(data):
    """Hiển thị thống kê về dữ liệu"""
    question_lengths = [len(item['question'].split()) for item in data if item['question']]
    answer_lengths = [len(item['answer'].split()) for item in data]

    print(f"Thống kê dữ liệu:")
    print(f"- Tổng số mục: {len(data)}")
    print(f"- Số mục có câu hỏi: {len([item for item in data if item['question']])}")
    if question_lengths:
        print(f"- Độ dài trung bình câu hỏi: {np.mean(question_lengths):.2f} từ")
        print(f"- Câu hỏi ngắn nhất: {min(question_lengths)} từ")
        print(f"- Câu hỏi dài nhất: {max(question_lengths)} từ")
    print(f"- Độ dài trung bình câu trả lời: {np.mean(answer_lengths):.2f} từ")
    print(f"- Câu trả lời ngắn nhất: {min(answer_lengths)} từ")
    print(f"- Câu trả lời dài nhất: {max(answer_lengths)} từ")

# Hàm tải mô hình embedding
def load_embedding_model(primary_model="vinai/phobert-base-v2", fallback_model="all-MiniLM-L6-v2"):
    """Tải mô hình embedding"""
    try:
        model = SentenceTransformer(primary_model)
        print(f"Đã tải mô hình {primary_model}")
        return model
    except Exception as e:
        print(f"Không thể tải mô hình {primary_model}, lỗi: {e}")
        print(f"Đang thử tải mô hình dự phòng...")
        model = SentenceTransformer(fallback_model)
        print(f"Đã tải mô hình dự phòng {fallback_model}")
        return model

# Hàm tạo embeddings
def create_embeddings(model, texts, batch_size=32):
    """Tạo embeddings từ texts"""
    print(f"Đang tạo embeddings cho {len(texts)} văn bản...")
    embeddings = model.encode(texts, batch_size=batch_size, show_progress_bar=True)
    return embeddings

# Hàm tạo documents
def create_documents(clean_data):
    """Tạo documents từ dữ liệu đã làm sạch"""
    documents = []
    for i, item in enumerate(clean_data):
        question = item['question']
        answer = item['answer']

        # Tạo nội dung kết hợp
        if question:
            content = f"Câu hỏi: {question}\nCâu trả lời: {answer}"
        else:
            content = f"Câu trả lời: {answer}"

        documents.append({
            'id': str(i),
            'question': question,
            'answer': answer,
            'content': content
        })

    return documents

# Xử lý dữ liệu chính
file_path = "/content/drive/MyDrive/datachatbot.json"  # Đường dẫn file trên Google Drive
data = load_data(file_path)

# Làm sạch dữ liệu
clean_data = process_data(data)

# Hiển thị thông tin thống kê
show_data_stats(clean_data)

# Tạo DataFrame để dễ xử lý 
df_clean = pd.DataFrame(clean_data)

# Tạo documents
documents = create_documents(clean_data)

# Tải mô hình embedding
model = load_embedding_model()

# Tạo embeddings cho documents
contents = [doc['content'] for doc in documents]
content_embeddings = create_embeddings(model, contents)

# Lưu documents và embeddings
with open('ptit_documents.pkl', 'wb') as f:
    pickle.dump(documents, f)

with open('ptit_embeddings.pkl', 'wb') as f:
    pickle.dump(content_embeddings, f)

print("Đã lưu documents và embeddings thành công!")

# Tải xuống các file đã tạo
files.download('ptit_documents.pkl')
files.download('ptit_embeddings.pkl')
print("Đã hoàn thành tiền xử lý dữ liệu!")

In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
import openai

# Thử tải từ file .env nếu có
try:
    load_dotenv()
    OPENROUTER_API_KEY = os.environ.get("OPENROUTER_API_KEY")
except:
    pass


os.environ["OPENAI_API_KEY"] = OPENROUTER_API_KEY

# Thiết lập base URL cho OpenRouter
openai.api_base = "https://openrouter.ai/api/v1"
os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"

# Các biến môi trường hữu ích cho OpenRouter
os.environ["ROUTER_REFERRER_URL"] = "https://ptit.edu.vn"
os.environ["ROUTER_TITLE"] = "PTIT Chatbot"

# Khởi tạo ChatOpenAI
llm = ChatOpenAI(
    model="mistralai/mistral-small-3.1-24b-instruct:free",
    temperature=0.3,
)

In [ ]:
# Tạo lại vector store từ file đã lưu
from langchain_community.vectorstores import FAISS
from langchain_core.embeddings import Embeddings  

# Tạo một wrapper class để sử dụng embeddings đã tạo sẵn
class PrecomputedEmbeddings(Embeddings):
    def __init__(self, model, documents, embeddings):
        self.model = model
        self.documents = documents
        self.embeddings = embeddings
        self.embedding_dict = {doc['content']: emb for doc, emb in zip(documents, embeddings)}

    def embed_documents(self, texts):
        # Lấy kích thước vector từ embedding đầu tiên
        vector_size = len(self.embeddings[0]) if len(self.embeddings) > 0 else 384
        return [self.embedding_dict.get(text, [0]*vector_size) for text in texts]

    def embed_query(self, text):
        # Sử dụng mô hình để embed truy vấn mới
        return self.model.encode(text).tolist()

# Tạo wrapper cho embeddings
embeddings_wrapper = PrecomputedEmbeddings(model, documents, content_embeddings)

# Tạo FAISS vector store
texts = [doc['content'] for doc in documents]
metadatas = [{"id": doc['id'], "question": doc['question'], "answer": doc['answer']} for doc in documents]

faiss_db = FAISS.from_texts(texts=texts, embedding=embeddings_wrapper, metadatas=metadatas)

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Tạo retriever từ vector store
retriever = faiss_db.as_retriever(search_kwargs={"k": 3})

# Tạo template cho prompt
template = """Bạn là một trợ lý AI giúp sinh viên và người quan tâm tìm hiểu về Học viện Công nghệ Bưu chính Viễn thông (PTIT).
Hãy sử dụng thông tin được cung cấp dưới đây để trả lời câu hỏi của người dùng một cách chính xác nhất.
Nếu không có đủ thông tin để trả lời, hãy thừa nhận điều đó và đề nghị liên hệ trực tiếp với nhà trường.

Thông tin tham khảo:
{context}

Câu hỏi: {question}

Trả lời bằng tiếng Việt, trình bày rõ ràng, mạch lạc và lịch sự:
"""

prompt = PromptTemplate.from_template(template)

def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

# Tạo RAG chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
import gradio as gr

def query_rag(query):
    if not query:
        return "Vui lòng nhập câu hỏi của bạn."

    try:
        result = rag_chain.invoke(query)
        return result
    except Exception as e:
        return f"Có lỗi xảy ra: {str(e)}"

# Tạo giao diện với Gradio
demo = gr.Interface(
    fn=query_rag,
    inputs=gr.Textbox(lines=2, placeholder="Nhập câu hỏi về PTIT..."),
    outputs="text",
    title="PTIT RAG Chatbot",
    description="Hỏi đáp về Học viện Công nghệ Bưu chính Viễn thông (PTIT)",
    examples=[
        ["PTIT có những ngành đào tạo nào?"],
        ["Học phí trung bình một năm ở PTIT là bao nhiêu?"],
        ["PTIT có ký túc xá cho sinh viên không?"],
        ["Điểm chuẩn ngành Công nghệ thông tin năm ngoái là bao nhiêu?"],
    ]
)

# Chạy demo
demo.launch(share=True)  